## Step 1: Setup Environment

In [ ]:
# For Google Colab
# from google.colab import drive
# drive.mount('/content/drive')
# !cp -r '/content/drive/MyDrive/majorproject - A' /content/project
# %cd /content/project

# For local setup
import sys
from pathlib import Path

# Add project to path
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import json
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

# Check GPU
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
print(f"MPS: {torch.backends.mps.is_available()}")

# Set device
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"GPU: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using MPS (Apple Silicon)")
else:
    device = torch.device('cpu')
    print("Using CPU")

## Step 2: Load Dataset

In [ ]:
# Load Telugu poems
data_path = Path('data/processed/telugu_poems.json')

if not data_path.exists():
    print("Dataset not found. Creating...")
    from scripts.create_large_dataset import main as create_dataset
    create_dataset()

with open(data_path, 'r', encoding='utf-8') as f:
    poems = json.load(f)

print(f"✅ Loaded {len(poems)} Telugu poems")

# Show sample
print("\n📝 Sample poem:")
print("-" * 50)
print(poems[0]['text'])

In [ ]:
# Dataset statistics
from collections import Counter

styles = Counter([p.get('style', 'unknown') for p in poems])
authors = Counter([p.get('author', 'unknown') for p in poems])

print("📊 Dataset Statistics:")
print(f"   Total poems: {len(poems)}")
print(f"\n   Styles:")
for style, count in styles.most_common(10):
    print(f"      {style}: {count}")
print(f"\n   Top Authors:")
for author, count in authors.most_common(5):
    print(f"      {author}: {count}")

## Step 3: Create Model V2

In [ ]:
from src.models.telugu_generator_v2 import create_telugu_generator_v2

# Create improved model
# Options: 'mbert' (recommended), 'distilmbert' (lighter), 'xlm-roberta' (heavier)
MODEL_TYPE = 'mbert'
FREEZE_ENCODER = False  # False = fine-tune encoder (better quality, slower)

print(f"🧠 Creating Telugu Generator V2 ({MODEL_TYPE})...")
model = create_telugu_generator_v2(
    model_type=MODEL_TYPE,
    freeze_encoder=FREEZE_ENCODER
)

model = model.to(device)

# Count parameters
total, trainable = model.count_parameters()
print(f"\n📊 Model Statistics:")
print(f"   Total parameters: {total:,}")
print(f"   Trainable: {trainable:,} ({100*trainable/total:.1f}%)")

## Step 4: Prepare Dataset

In [ ]:
from src.preprocessing.telugu_cleaner import TeluguTextCleaner
import random

class TeluguPoemDatasetV2(Dataset):
    """Improved dataset with augmentation and proper label creation."""
    
    def __init__(self, poems, tokenizer, max_length=128, augment=True):
        self.poems = poems
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.augment = augment
        self.cleaner = TeluguTextCleaner()
        self.data = self._prepare_data()
    
    def _prepare_data(self):
        data = []
        for poem in self.poems:
            text = poem.get('text', '') if isinstance(poem, dict) else poem
            text = self.cleaner.clean(text)
            if len(text) > 10:
                data.append(text)
                # Add individual lines for more training data
                lines = text.split('\n')
                for line in lines:
                    if len(line.strip()) > 10:
                        data.append(line.strip())
        return data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text = self.data[idx]
        
        # Simple augmentation: random word shuffling
        if self.augment and random.random() < 0.15:
            words = text.split()
            if len(words) > 3:
                i = random.randint(1, len(words)-2)
                words[i], words[i+1] = words[i+1], words[i]
                text = ' '.join(words)
        
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        
        # Create labels (same as input, pad tokens masked)
        labels = input_ids.clone()
        labels[labels == self.tokenizer.pad_token_id] = -100
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

# Create dataset
dataset = TeluguPoemDatasetV2(
    poems, 
    model.tokenizer, 
    max_length=128,
    augment=True
)

# Split into train/val
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

print(f"✅ Dataset created:")
print(f"   Total: {len(dataset)}")
print(f"   Train: {len(train_dataset)}")
print(f"   Val: {len(val_dataset)}")

In [ ]:
# Create dataloaders
BATCH_SIZE = 4  # Adjust based on your GPU memory

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    pin_memory=True if device.type == 'cuda' else False
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

print(f"✅ DataLoaders ready:")
print(f"   Train batches: {len(train_loader)}")
print(f"   Val batches: {len(val_loader)}")

## Step 5: Training with Anti-Repetition Losses

In [ ]:
# Training configuration
CONFIG = {
    'epochs': 20,
    'learning_rate': 2e-5,
    'weight_decay': 0.01,
    'warmup_steps': 200,
    'max_grad_norm': 1.0,
    'accumulation_steps': 4,  # Effective batch size = 16
    'log_every': 50,
    'eval_every': 2,  # epochs
    'save_every': 5,  # epochs
    # Anti-repetition weights
    'diversity_weight': 0.1,
    'repetition_weight': 0.2
}

print("📋 Training Configuration:")
for k, v in CONFIG.items():
    print(f"   {k}: {v}")

In [ ]:
from torch.optim import AdamW
import math

# Optimizer with weight decay
no_decay = ['bias', 'LayerNorm.weight']
optimizer_params = [
    {'params': [p for n, p in model.named_parameters() 
                if not any(nd in n for nd in no_decay) and p.requires_grad],
     'weight_decay': CONFIG['weight_decay']},
    {'params': [p for n, p in model.named_parameters() 
                if any(nd in n for nd in no_decay) and p.requires_grad],
     'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_params, lr=CONFIG['learning_rate'])

# Learning rate scheduler with warmup
total_steps = len(train_loader) * CONFIG['epochs'] // CONFIG['accumulation_steps']

def get_lr(step):
    if step < CONFIG['warmup_steps']:
        return CONFIG['learning_rate'] * (step + 1) / CONFIG['warmup_steps']
    else:
        progress = (step - CONFIG['warmup_steps']) / max(1, total_steps - CONFIG['warmup_steps'])
        return CONFIG['learning_rate'] * (1 + math.cos(math.pi * progress)) / 2

print(f"Total training steps: {total_steps}")
print(f"Warmup steps: {CONFIG['warmup_steps']}")

In [ ]:
# Custom diversity loss to prevent repetition
class DiversityLoss(nn.Module):
    def __init__(self, weight=0.1):
        super().__init__()
        self.weight = weight
    
    def forward(self, logits):
        # Encourage diverse predictions by maximizing entropy
        probs = F.softmax(logits, dim=-1)
        entropy = -(probs * torch.log(probs + 1e-10)).sum(dim=-1)
        # We want some entropy (diversity) but not too much (quality)
        # Target entropy around 2-3
        target_entropy = 2.5
        loss = F.relu(target_entropy - entropy.mean())
        return self.weight * loss

class RepetitionPenalty(nn.Module):
    def __init__(self, weight=0.2, window=10):
        super().__init__()
        self.weight = weight
        self.window = window
    
    def forward(self, logits, input_ids):
        batch_size, seq_len, vocab_size = logits.shape
        loss = torch.tensor(0.0, device=logits.device)
        
        for t in range(self.window, seq_len):
            recent = input_ids[:, t-self.window:t]
            probs = F.softmax(logits[:, t, :], dim=-1)
            
            for b in range(batch_size):
                for token in recent[b].unique():
                    if token != 0:  # Skip padding
                        loss = loss + probs[b, token]
        
        return self.weight * loss / (batch_size * max(1, seq_len - self.window))

diversity_loss_fn = DiversityLoss(CONFIG['diversity_weight'])
repetition_loss_fn = RepetitionPenalty(CONFIG['repetition_weight'])

In [ ]:
# Training loop
from pathlib import Path

save_dir = Path('checkpoints/telugu_v2')
save_dir.mkdir(parents=True, exist_ok=True)

# Test prompts for evaluation
test_prompts = [
    "చందమామ రావే",
    "తెలుగు భాష మన",
    "అమ్మ ప్రేమ"
]

# Tracking
train_losses = []
val_losses = []
best_loss = float('inf')
global_step = 0

print("\n" + "="*60)
print("🚀 Starting Telugu Poem Training V2")
print("="*60)

for epoch in range(1, CONFIG['epochs'] + 1):
    # Training
    model.train()
    epoch_loss = 0
    num_batches = 0
    
    progress = tqdm(train_loader, desc=f"Epoch {epoch}/{CONFIG['epochs']}")
    optimizer.zero_grad()
    
    for batch_idx, batch in enumerate(progress):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Forward
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs['loss']
        if loss is None:
            continue
        
        # Add anti-repetition losses
        if outputs.get('logits') is not None:
            loss = loss + diversity_loss_fn(outputs['logits'])
            loss = loss + repetition_loss_fn(outputs['logits'], input_ids)
        
        # Scale for accumulation
        loss = loss / CONFIG['accumulation_steps']
        loss.backward()
        
        # Update weights
        if (batch_idx + 1) % CONFIG['accumulation_steps'] == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), CONFIG['max_grad_norm'])
            
            # Update LR
            lr = get_lr(global_step)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
        
        epoch_loss += loss.item() * CONFIG['accumulation_steps']
        num_batches += 1
        
        progress.set_postfix({'loss': f'{loss.item() * CONFIG["accumulation_steps"]:.4f}', 'lr': f'{get_lr(global_step):.2e}'})
    
    avg_train_loss = epoch_loss / num_batches
    train_losses.append(avg_train_loss)
    print(f"\n📊 Epoch {epoch} | Train Loss: {avg_train_loss:.4f}")
    
    # Validation
    if epoch % CONFIG['eval_every'] == 0:
        model.eval()
        val_loss = 0
        num_val = 0
        
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                if outputs['loss'] is not None:
                    val_loss += outputs['loss'].item()
                    num_val += 1
        
        avg_val_loss = val_loss / max(1, num_val)
        val_losses.append(avg_val_loss)
        print(f"   Val Loss: {avg_val_loss:.4f}")
    
    # Test generation
    if epoch % CONFIG['eval_every'] == 0:
        print("\n📝 Generation Test:")
        model.eval()
        for prompt in test_prompts:
            try:
                generated = model.generate(
                    prompt,
                    max_length=40,
                    temperature=0.8,
                    repetition_penalty=1.5,
                    no_repeat_ngram_size=3
                )
                print(f"   {prompt} → {generated[:80]}...")
            except Exception as e:
                print(f"   Error: {e}")
    
    # Save best model
    if avg_train_loss < best_loss:
        best_loss = avg_train_loss
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'epoch': epoch,
            'loss': best_loss
        }, save_dir / 'best_model.pt')
        print(f"   ⭐ Best model saved!")
    
    # Regular checkpoint
    if epoch % CONFIG['save_every'] == 0:
        torch.save({
            'model_state_dict': model.state_dict(),
            'epoch': epoch,
            'loss': avg_train_loss
        }, save_dir / f'checkpoint_epoch_{epoch}.pt')

print("\n" + "="*60)
print("✅ Training Complete!")
print(f"   Best Loss: {best_loss:.4f}")
print("="*60)

## Step 6: Evaluate Results

In [ ]:
# Plot training curves
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()
plt.grid(True)

if val_losses:
    plt.subplot(1, 2, 2)
    plt.plot(val_losses, label='Val Loss', color='orange')
    plt.xlabel('Evaluation Step')
    plt.ylabel('Loss')
    plt.title('Validation Loss')
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.savefig(save_dir / 'training_curves.png', dpi=150)
plt.show()

In [ ]:
# Comprehensive generation test
model.eval()

test_prompts = [
    "చందమామ రావే",
    "తెలుగు భాష మన",
    "అమ్మ ప్రేమ గొప్పది",
    "నా దేశం భారతం",
    "పూలు పూయగ"
]

print("\n" + "="*60)
print("📝 Telugu Poem Generation Test")
print("="*60)

# Test different sampling strategies
strategies = [
    {'name': 'Nucleus (p=0.9)', 'top_p': 0.9, 'top_k': 0, 'temp': 0.8},
    {'name': 'Top-k (k=50)', 'top_p': 1.0, 'top_k': 50, 'temp': 0.8},
    {'name': 'High Temp (1.2)', 'top_p': 0.95, 'top_k': 0, 'temp': 1.2}
]

for prompt in test_prompts:
    print(f"\n🔹 Prompt: {prompt}")
    print("-" * 50)
    
    for strategy in strategies:
        try:
            generated = model.generate(
                prompt,
                max_length=50,
                temperature=strategy['temp'],
                top_k=strategy['top_k'],
                top_p=strategy['top_p'],
                repetition_penalty=1.5,
                no_repeat_ngram_size=3
            )
            print(f"   [{strategy['name']}]: {generated}")
        except Exception as e:
            print(f"   [{strategy['name']}]: Error - {e}")

In [ ]:
# Analyze repetition in generated text
def analyze_repetition(text):
    """Analyze repetition metrics in generated text."""
    words = text.split()
    
    if len(words) < 2:
        return {'consecutive_rep': 0, 'unique_ratio': 1.0, 'bigram_rep': 0}
    
    # Consecutive repetition
    consecutive = sum(1 for i in range(1, len(words)) if words[i] == words[i-1])
    
    # Unique word ratio
    unique_ratio = len(set(words)) / len(words)
    
    # Bigram repetition
    bigrams = [(words[i], words[i+1]) for i in range(len(words)-1)]
    bigram_rep = 1 - len(set(bigrams)) / max(1, len(bigrams))
    
    return {
        'consecutive_rep': consecutive / len(words),
        'unique_ratio': unique_ratio,
        'bigram_rep': bigram_rep
    }

# Generate samples and analyze
print("\n📊 Repetition Analysis:")
print("="*60)

for prompt in test_prompts[:3]:
    generated = model.generate(
        prompt,
        max_length=50,
        temperature=0.8,
        repetition_penalty=1.5,
        no_repeat_ngram_size=3
    )
    
    metrics = analyze_repetition(generated)
    
    print(f"\nPrompt: {prompt}")
    print(f"Generated: {generated[:100]}...")
    print(f"Metrics:")
    print(f"  - Consecutive repetition: {metrics['consecutive_rep']:.2%}")
    print(f"  - Unique word ratio: {metrics['unique_ratio']:.2%}")
    print(f"  - Bigram repetition: {metrics['bigram_rep']:.2%}")

## Step 7: Save Final Model

In [ ]:
# Save final model with config
final_save = {
    'model_state_dict': model.state_dict(),
    'config': CONFIG,
    'model_type': MODEL_TYPE,
    'best_loss': best_loss,
    'train_losses': train_losses,
    'val_losses': val_losses
}

torch.save(final_save, save_dir / 'final_model.pt')

print(f"✅ Final model saved to: {save_dir / 'final_model.pt'}")
print(f"\n📋 To load the model:")
print(f"   checkpoint = torch.load('{save_dir}/final_model.pt')")
print(f"   model.load_state_dict(checkpoint['model_state_dict'])")

## 💡 Tips for Better Generation

If you're still seeing repetition:

1. **Increase `repetition_penalty`** (try 1.5 - 2.0)
2. **Increase `no_repeat_ngram_size`** (try 4-5)
3. **Lower `temperature`** for more focused output (0.6-0.8)
4. **Use nucleus sampling** with `top_p=0.9`
5. **Train for more epochs** (30-50)
6. **Use larger dataset** if possible
7. **Unfreeze encoder** for better Telugu understanding